# Chapter 14: Middleware & Proxy

Middleware in Next.js provides a powerful way to intercept and modify requests before they reach your application pages or API routes. Whether you're implementing authentication checks, handling redirects, or configuring edge-based proxies, middleware runs at the edge for minimal latency and maximum performance.

By the end of this chapter, you'll master creating middleware functions, implementing authentication guards, managing redirects and rewrites, leveraging the new Proxy configuration features, and optimizing for the Edge Runtime.

## 14.1 Understanding Middleware

Middleware functions execute before a request completes, allowing you to modify the response or redirect the user based on specific conditions.

### Basic Middleware Structure

Create a `middleware.ts` (or `middleware.js`) file in your project root:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  // Log requests for analytics
  console.log(`[${new Date().toISOString()}] ${request.method} ${request.url}`);
  
  // Continue to the requested page
  return NextResponse.next();
}

// Configure which routes middleware applies to
export const config = {
  matcher: ['/((?!api|_next/static|_next/image|favicon.ico).*)'],
};
```

### Middleware Execution Context

Middleware runs in the Edge Runtime, which has specific constraints and capabilities:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export async function middleware(request: NextRequest) {
  // Access request properties
  const { pathname, searchParams } = request.nextUrl;
  const userAgent = request.headers.get('user-agent') || '';
  const ip = request.ip || 'unknown';
  
  // Example: Block old browsers
  if (userAgent.includes('MSIE') || userAgent.includes('Trident')) {
    return new NextResponse('Browser not supported', { status: 400 });
  }
  
  // Example: Add custom headers to all responses
  const response = NextResponse.next();
  response.headers.set('x-request-id', crypto.randomUUID());
  response.headers.set('x-middleware-processed', 'true');
  
  return response;
}
```

### Conditional Middleware Execution

Apply middleware selectively using the matcher configuration:

```typescript
// middleware.ts
export const config = {
  matcher: [
    /*
     * Match all request paths except for the ones starting with:
     * - api (API routes)
     * - _next/static (static files)
     * - _next/image (image optimization files)
     * - favicon.ico (favicon file)
     */
    '/((?!api|_next/static|_next/image|favicon.ico).*)',
    
    // Or use specific patterns:
    '/dashboard/:path*',     // Match /dashboard and all subpaths
    '/admin/:path*',         // Match /admin and all subpaths
    '/((?!public).*)',       // Exclude public directory
  ],
};
```

## 14.2 Request/Response Interception

Modify requests and responses on the fly for A/B testing, localization, or custom headers.

### Modifying Request Headers

Inject headers before the request reaches your application:

```typescript
// middleware.ts
import { NextRequest, NextResponse } from 'next/server';

export function middleware(request: NextRequest) {
  // Clone the request headers
  const requestHeaders = new Headers(request.headers);
  
  // Add custom headers
  requestHeaders.set('x-forwarded-host', request.headers.get('host') || '');
  requestHeaders.set('x-forwarded-proto', request.nextUrl.protocol);
  requestHeaders.set('x-request-start', Date.now().toString());
  
  // Create new response with modified headers
  const response = NextResponse.next({
    request: {
      headers: requestHeaders,
    },
  });
  
  return response;
}
```

### Geolocation and Personalization

Use geo data to customize content:

```typescript
// middleware.ts
import { NextRequest, NextResponse } from 'next/server';

export function middleware(request: NextRequest) {
  const { geo } = request;
  
  // Add geo data to headers for use in server components
  const requestHeaders = new Headers(request.headers);
  
  if (geo) {
    requestHeaders.set('x-country', geo.country || 'unknown');
    requestHeaders.set('x-city', geo.city || 'unknown');
    requestHeaders.set('x-region', geo.region || 'unknown');
    requestHeaders.set('x-latitude', geo.latitude || '0');
    requestHeaders.set('x-longitude', geo.longitude || '0');
  }
  
  // Check for timezone
  const timezone = requestHeaders.get('x-vercel-ip-timezone') || 'UTC';
  requestHeaders.set('x-timezone', timezone);
  
  return NextResponse.next({
    request: { headers: requestHeaders },
  });
}

// Usage in server component:
// headers().get('x-country') to access location
```

### Response Modification

Transform responses before they reach the client:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  const response = NextResponse.next();
  
  // Add security headers to all responses
  response.headers.set('X-DNS-Prefetch-Control', 'on');
  response.headers.set('Strict-Transport-Security', 'max-age=63072000; includeSubDomains; preload');
  response.headers.set('X-XSS-Protection', '1; mode=block');
  response.headers.set('X-Frame-Options', 'SAMEORIGIN');
  response.headers.set('X-Content-Type-Options', 'nosniff');
  response.headers.set('Referrer-Policy', 'origin-when-cross-origin');
  
  // Add custom debugging headers in development
  if (process.env.NODE_ENV === 'development') {
    response.headers.set('x-middleware-cache', 'no-cache');
  }
  
  return response;
}
```

## 14.3 Authentication Guards

Implement authentication checks and protected routes without cluttering your page components.

### Basic Authentication Check

Protect routes based on session status:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';
import { verifyAuth } from '@/lib/auth';

export async function middleware(request: NextRequest) {
  const { pathname } = request.nextUrl;
  
  // Define protected routes
  const protectedPaths = ['/dashboard', '/profile', '/settings'];
  const isProtectedPath = protectedPaths.some(path => 
    pathname.startsWith(path)
  );
  
  if (!isProtectedPath) {
    return NextResponse.next();
  }
  
  // Check for auth token
  const token = request.cookies.get('auth-token')?.value;
  
  if (!token) {
    const loginUrl = new URL('/login', request.url);
    loginUrl.searchParams.set('from', pathname);
    return NextResponse.redirect(loginUrl);
  }
  
  try {
    // Verify token
    const payload = await verifyAuth(token);
    
    if (!payload) {
      throw new Error('Invalid token');
    }
    
    // Add user info to headers for downstream use
    const requestHeaders = new Headers(request.headers);
    requestHeaders.set('x-user-id', payload.userId);
    requestHeaders.set('x-user-role', payload.role);
    
    return NextResponse.next({
      request: { headers: requestHeaders },
    });
  } catch (error) {
    // Clear invalid cookie and redirect
    const response = NextResponse.redirect(new URL('/login', request.url));
    response.cookies.delete('auth-token');
    return response;
  }
}

export const config = {
  matcher: ['/dashboard/:path*', '/profile/:path*', '/settings/:path*'],
};
```

### Role-Based Access Control

Implement granular permissions in middleware:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

type RoutePermissions = {
  [key: string]: string[];
};

const routePermissions: RoutePermissions = {
  '/admin': ['admin'],
  '/admin/users': ['admin'],
  '/dashboard/analytics': ['admin', 'analyst'],
  '/api/admin': ['admin'],
};

export async function middleware(request: NextRequest) {
  const { pathname } = request.nextUrl;
  
  // Find matching route pattern
  const matchedRoute = Object.keys(routePermissions).find(route => 
    pathname.startsWith(route)
  );
  
  if (!matchedRoute) {
    return NextResponse.next();
  }
  
  const requiredRoles = routePermissions[matchedRoute];
  const token = request.cookies.get('auth-token')?.value;
  
  if (!token) {
    return NextResponse.redirect(new URL('/login', request.url));
  }
  
  try {
    const payload = await verifyAuth(token);
    const userRole = payload.role;
    
    if (!requiredRoles.includes(userRole)) {
      // User doesn't have permission
      return NextResponse.rewrite(new URL('/403', request.url));
    }
    
    // Add role to headers for potential use in API routes
    const requestHeaders = new Headers(request.headers);
    requestHeaders.set('x-user-role', userRole);
    
    return NextResponse.next({
      request: { headers: requestHeaders },
    });
  } catch {
    return NextResponse.redirect(new URL('/login', request.url));
  }
}
```

### API Route Protection

Secure API endpoints using middleware:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export async function middleware(request: NextRequest) {
  // Only process API routes
  if (!request.nextUrl.pathname.startsWith('/api/')) {
    return NextResponse.next();
  }
  
  // Rate limiting check
  const ip = request.ip ?? '127.0.0.1';
  const isRateLimited = await checkRateLimit(ip);
  
  if (isRateLimited) {
    return NextResponse.json(
      { error: 'Too many requests' },
      { status: 429, headers: { 'Retry-After': '60' } }
    );
  }
  
  // API key validation for external access
  if (request.nextUrl.pathname.startsWith('/api/external')) {
    const apiKey = request.headers.get('x-api-key');
    
    if (!apiKey || !(await validateApiKey(apiKey))) {
      return NextResponse.json(
        { error: 'Unauthorized' },
        { status: 401 }
      );
    }
  }
  
  return NextResponse.next();
}

async function checkRateLimit(ip: string): Promise<boolean> {
  // Implementation using Redis or similar
  return false;
}

async function validateApiKey(key: string): Promise<boolean> {
  // Validate against database
  return true;
}

export const config = {
  matcher: '/api/:path*',
};
```

## 14.4 Redirection and Rewriting

Handle URL transformations for SEO, legacy URLs, or multi-tenant applications.

### URL Redirects

Implement permanent and temporary redirects:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  const { pathname, search } = request.nextUrl;
  
  // Legacy URL redirects
  const redirects = [
    { from: '/old-about', to: '/about', permanent: true },
    { from: '/products/:id', to: '/shop/:id', permanent: true },
    { from: '/blog.php', to: '/blog', permanent: true },
  ];
  
  for (const redirect of redirects) {
    if (pathname === redirect.from) {
      const url = request.nextUrl.clone();
      url.pathname = redirect.to;
      
      return NextResponse.redirect(url, redirect.permanent ? 308 : 307);
    }
  }
  
  // WWW to non-WWW redirect (or vice versa)
  const host = request.headers.get('host') || '';
  if (host.startsWith('www.')) {
    const newHost = host.replace('www.', '');
    const url = request.nextUrl.clone();
    url.host = newHost;
    return NextResponse.redirect(url, 301);
  }
  
  // HTTP to HTTPS redirect
  if (process.env.NODE_ENV === 'production' && request.nextUrl.protocol === 'http:') {
    const url = request.nextUrl.clone();
    url.protocol = 'https:';
    return NextResponse.redirect(url, 301);
  }
  
  return NextResponse.next();
}
```

### URL Rewriting

Rewrite URLs internally without changing the browser address:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  const { pathname } = request.nextUrl;
  
  // Multi-tenant application: Rewrite /[tenant]/dashboard to /dashboard
  const tenantMatch = pathname.match(/^\/([^/]+)\/dashboard/);
  if (tenantMatch) {
    const tenant = tenantMatch[1];
    const url = request.nextUrl.clone();
    url.pathname = `/dashboard${pathname.replace(`/${tenant}/dashboard`, '')}`;
    
    // Add tenant to headers for context
    const response = NextResponse.rewrite(url);
    response.headers.set('x-tenant-id', tenant);
    return response;
  }
  
  // A/B Testing: Rewrite 50% of users to variant B
  if (pathname === '/landing') {
    const variant = request.cookies.get('ab-test-variant')?.value;
    
    if (!variant) {
      // Assign variant if not set
      const newVariant = Math.random() > 0.5 ? 'b' : 'a';
      const url = request.nextUrl.clone();
      url.pathname = newVariant === 'b' ? '/landing-variant-b' : '/landing';
      
      const response = NextResponse.rewrite(url);
      response.cookies.set('ab-test-variant', newVariant, {
        maxAge: 60 * 60 * 24 * 7, // 1 week
      });
      return response;
    }
    
    if (variant === 'b') {
      const url = request.nextUrl.clone();
      url.pathname = '/landing-variant-b';
      return NextResponse.rewrite(url);
    }
  }
  
  // Internationalization fallback
  if (pathname.startsWith('/old-i18n/')) {
    const newPath = pathname.replace('/old-i18n/', '/');
    return NextResponse.rewrite(new URL(newPath, request.url));
  }
  
  return NextResponse.next();
}
```

### Multi-Language Redirects

Handle locale-based routing:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';
import { match } from '@formatjs/intl-localematcher';
import Negotiator from 'negotiator';

const locales = ['en-US', 'en', 'fr-FR', 'fr', 'de-DE', 'de'];
const defaultLocale = 'en-US';

function getLocale(request: NextRequest): string {
  const negotiatorHeaders: Record<string, string> = {};
  request.headers.forEach((value, key) => (negotiatorHeaders[key] = value));
  
  const languages = new Negotiator({ headers: negotiatorHeaders }).languages();
  return match(languages, locales, defaultLocale);
}

export function middleware(request: NextRequest) {
  const { pathname } = request.nextUrl;
  
  // Check if pathname already has a locale
  const pathnameHasLocale = locales.some(
    locale => pathname.startsWith(`/${locale}/`) || pathname === `/${locale}`
  );
  
  if (pathnameHasLocale) return NextResponse.next();
  
  // Redirect to locale-specific path
  const locale = getLocale(request);
  request.nextUrl.pathname = `/${locale}${pathname}`;
  
  return NextResponse.redirect(request.nextUrl);
}

export const config = {
  matcher: ['/((?!api|_next/static|_next/image|favicon.ico).*)'],
};
```

## 14.5 Proxy Configuration (New in Next.js 16)

Next.js 16 introduces enhanced proxy capabilities in middleware for advanced request forwarding.

### Basic Proxy Setup

Forward requests to external services:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export async function middleware(request: NextRequest) {
  // Proxy API requests to external service
  if (request.nextUrl.pathname.startsWith('/api/external/')) {
    const targetUrl = new URL(
      request.nextUrl.pathname.replace('/api/external/', ''),
      'https://api.external-service.com'
    );
    
    // Copy search params
    targetUrl.search = request.nextUrl.search;
    
    // Forward request with auth
    const response = await fetch(targetUrl, {
      method: request.method,
      headers: {
        ...Object.fromEntries(request.headers),
        'Authorization': `Bearer ${process.env.EXTERNAL_API_KEY}`,
        'Host': 'api.external-service.com',
      },
      body: request.body,
    });
    
    // Return proxied response
    return new NextResponse(response.body, {
      status: response.status,
      statusText: response.statusText,
      headers: response.headers,
    });
  }
  
  return NextResponse.next();
}
```

### Path-Based Proxy Routing

Route different paths to different backend services:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

const services = {
  '/api/payments': 'https://payments.internal.com',
  '/api/inventory': 'https://inventory.internal.com',
  '/api/analytics': 'https://analytics.internal.com',
};

export async function middleware(request: NextRequest) {
  const { pathname } = request.nextUrl;
  
  // Find matching service
  const servicePath = Object.keys(services).find(path => 
    pathname.startsWith(path)
  );
  
  if (servicePath) {
    const targetBase = services[servicePath as keyof typeof services];
    const targetPath = pathname.replace(servicePath, '');
    const targetUrl = new URL(targetPath + request.nextUrl.search, targetBase);
    
    try {
      const response = await fetch(targetUrl, {
        method: request.method,
        headers: {
          'Content-Type': 'application/json',
          'X-Forwarded-For': request.ip || 'unknown',
          'X-Original-Host': request.headers.get('host') || '',
        },
        body: ['GET', 'HEAD'].includes(request.method) ? null : await request.text(),
      });
      
      // Transform response if needed
      const data = await response.json();
      
      return NextResponse.json(data, {
        status: response.status,
        headers: {
          'X-Proxied-By': 'nextjs-middleware',
        },
      });
    } catch (error) {
      return NextResponse.json(
        { error: 'Service unavailable' },
        { status: 503 }
      );
    }
  }
  
  return NextResponse.next();
}
```

### WebSocket Proxying

Handle WebSocket upgrades through middleware:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  const upgradeHeader = request.headers.get('upgrade');
  
  if (upgradeHeader === 'websocket') {
    // Rewrite WebSocket connections to external server
    if (request.nextUrl.pathname.startsWith('/ws/')) {
      const targetUrl = new URL(
        request.nextUrl.pathname,
        'wss://realtime-backend.com'
      );
      
      // Note: Actual WebSocket proxying requires platform support
      // This sets headers for the upgrade
      const response = NextResponse.next();
      response.headers.set('x-websocket-proxy', targetUrl.toString());
      return response;
    }
  }
  
  return NextResponse.next();
}
```

## 14.6 Edge Runtime Considerations

Middleware runs in the Edge Runtime, which has specific constraints compared to Node.js.

### Edge Runtime Limitations

Understand what's available and what's not:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export async function middleware(request: NextRequest) {
  // ✅ Available in Edge Runtime:
  // - fetch API
  // - Request/Response objects
  // - Web Crypto API
  // - Encoding APIs (TextEncoder, TextDecoder)
  // - URL and URLSearchParams
  
  const data = await fetch('https://api.example.com/data').then(r => r.json());
  const encoder = new TextEncoder();
  const encoded = encoder.encode(JSON.stringify(data));
  
  // ❌ Not available in Edge Runtime:
  // - Node.js built-in modules (fs, path, crypto, etc.)
  // - Database drivers that require Node.js (unless using Edge-compatible drivers)
  // - File system operations
  
  // Workaround: Use fetch to call API routes for Node.js operations
  if (requiresDatabaseOperation(request)) {
    const result = await fetch(`${request.nextUrl.origin}/api/edge-proxy`, {
      method: 'POST',
      body: JSON.stringify({ action: 'check-db', data: request.url }),
    });
    
    if (!result.ok) {
      return NextResponse.json({ error: 'DB check failed' }, { status: 500 });
    }
  }
  
  return NextResponse.next();
}
```

### Optimizing Edge Performance

Keep middleware lightweight for optimal edge performance:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

// ❌ Bad: Heavy computation in middleware
export async function middleware(request: NextRequest) {
  // Don't do this - blocks the request
  const heavyData = await fetchLargeDataset();
  const processed = heavyData.map(item => expensiveTransform(item));
  
  return NextResponse.next();
}

// ✅ Good: Lightweight checks, defer heavy work
export function middleware(request: NextRequest) {
  // Quick checks only
  const token = request.cookies.get('token')?.value;
  
  if (!token && isProtectedRoute(request.nextUrl.pathname)) {
    return NextResponse.redirect(new URL('/login', request.url));
  }
  
  // Add header to let page know auth status, but don't fetch user data
  const response = NextResponse.next();
  response.headers.set('x-auth-status', token ? 'authenticated' : 'anonymous');
  
  return response;
}
```

### Conditional Edge vs Node.js

Route to different handlers based on requirements:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  const { pathname } = request.nextUrl;
  
  // Route complex operations to Node.js API routes
  if (pathname.startsWith('/api/complex-report')) {
    // This API route runs in Node.js runtime, not Edge
    return NextResponse.next();
  }
  
  // Keep simple operations at edge
  if (pathname.startsWith('/api/simple')) {
    // Add edge-specific headers
    const response = NextResponse.next();
    response.headers.set('x-edge-processed', 'true');
    return response;
  }
  
  return NextResponse.next();
}

// Force Node.js runtime for specific API routes
// app/api/complex-report/route.ts
export const runtime = 'nodejs'; // Force Node.js runtime

export async function POST(request: Request) {
  // Can use Node.js specific modules here
  const fs = await import('fs');
  // ... heavy processing
}
```

## 14.7 Common Middleware Patterns

Reusable patterns for common middleware use cases.

### Maintenance Mode

Toggle maintenance mode via environment variable:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  // Check for maintenance mode
  if (process.env.MAINTENANCE_MODE === 'true') {
    // Allow admin users to bypass
    const adminToken = request.cookies.get('admin-bypass')?.value;
    if (adminToken === process.env.MAINTENANCE_BYPASS_TOKEN) {
      return NextResponse.next();
    }
    
    // Redirect to maintenance page
    if (request.nextUrl.pathname !== '/maintenance') {
      return NextResponse.redirect(new URL('/maintenance', request.url));
    }
  }
  
  return NextResponse.next();
}
```

### Bot Detection and Handling

Identify and handle bot traffic:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  const userAgent = request.headers.get('user-agent') || '';
  
  // Common bot patterns
  const botPatterns = [
    /bot/i, /crawler/i, /spider/i, /scrape/i,
    /facebookexternalhit/i, /twitterbot/i, /linkedinbot/i,
  ];
  
  const isBot = botPatterns.some(pattern => pattern.test(userAgent));
  
  if (isBot) {
    const response = NextResponse.next();
    response.headers.set('x-is-bot', 'true');
    
    // Optional: Serve prerendered content for bots
    if (request.nextUrl.pathname.startsWith('/blog/')) {
      // Rewrite to static version for SEO bots
      const url = request.nextUrl.clone();
      url.pathname = `/static-prerender${request.nextUrl.pathname}`;
      return NextResponse.rewrite(url);
    }
  }
  
  return NextResponse.next();
}
```

### Request Timing and Logging

Track request performance:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  const start = Date.now();
  
  const response = NextResponse.next();
  
  // Calculate duration
  const duration = Date.now() - start;
  
  // Add timing header
  response.headers.set('x-response-time', `${duration}ms`);
  
  // Async logging (don't await to avoid blocking)
  logRequest({
    method: request.method,
    path: request.nextUrl.pathname,
    duration,
    userAgent: request.headers.get('user-agent'),
    ip: request.ip,
    timestamp: new Date().toISOString(),
  });
  
  return response;
}

async function logRequest(data: any) {
  // Send to logging service
  try {
    await fetch(process.env.LOG_ENDPOINT || '', {
      method: 'POST',
      body: JSON.stringify(data),
    });
  } catch {
    // Silent fail for logging
  }
}
```

### Feature Flags

Toggle features dynamically:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

// Simple feature flag check (could be from Redis or API in production)
const featureFlags = {
  'new-dashboard': true,
  'beta-feature': false,
};

export function middleware(request: NextRequest) {
  const { pathname } = request.nextUrl;
  
  // Check if accessing beta feature
  if (pathname.startsWith('/beta/')) {
    if (!featureFlags['beta-feature']) {
      return NextResponse.redirect(new URL('/coming-soon', request.url));
    }
    
    // Add beta header for frontend awareness
    const response = NextResponse.next();
    response.headers.set('x-beta-enabled', 'true');
    return response;
  }
  
  // Route to new dashboard if feature enabled
  if (pathname === '/dashboard' && featureFlags['new-dashboard']) {
    return NextResponse.rewrite(new URL('/dashboard-v2', request.url));
  }
  
  return NextResponse.next();
}
```

## Key Takeaways from Chapter 14

1. **Middleware Fundamentals**: Middleware runs before every request (matching your config) in the Edge Runtime, allowing you to intercept requests, modify responses, redirect users, and add security headers with minimal latency.

2. **Request/Response Interception**: Use `NextRequest` and `NextResponse` to read request headers, cookies, and geo-data. Inject custom headers into requests for downstream use in server components or API routes.

3. **Authentication Guards**: Implement centralized authentication checks in middleware to protect route groups, validate JWT tokens, check user roles, and redirect unauthenticated users—keeping your page components clean of auth logic.

4. **Redirection and Rewriting**: Use `NextResponse.redirect()` for external navigation (301/307 status codes) and `NextResponse.rewrite()` for internal URL mapping without changing the browser address—ideal for A/B testing, multi-tenancy, and legacy URL support.

5. **Proxy Configuration**: Next.js 16 enhances middleware proxying capabilities, allowing you to forward requests to external services, route different paths to different microservices, and handle WebSocket upgrades at the edge.

6. **Edge Runtime Constraints**: Middleware runs in a lightweight Edge Runtime with limited Node.js APIs. Use standard Web APIs (fetch, crypto, encoding), avoid heavy computation, and delegate Node.js-specific tasks (database queries, file system) to API routes with `export const runtime = 'nodejs'`.

7. **Common Patterns**: Implement maintenance mode switches, bot detection for SEO optimization, request timing/logging for performance monitoring, and feature flag systems to gradually roll out functionality without redeploying.

## Coming Up Next

**Chapter 15: State Management**

Now that you can control request flow with middleware, it's time to master managing data across your application. In Chapter 15, we'll explore React Context API, server-side state patterns, client-side libraries like Zustand and Redux, URL state management, and optimistic updates. You'll learn when to use each approach and how to synchronize state between server and client components effectively.